# <span style="color: blue">Laboratorio 6: Estimación I</span> <a class="tocSkip"> 

### <span style="color: green">Prof. Andrés Arguedas - XS-0300 - II 2020</span> <a class="tocSkip"> 

*Este laboratorio es parte del curso XS-0300 Estadística para Biología I de la Escuela de Estadística de la Universidad de Costa Rica, y se presenta con una licencia [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/).*

**ATENCIÓN:** Antes de entregar este laboratorio, asegúrese que todo corre como se espera. Para hacer esto, primero **reinice el kernel**, esto lo puede hacer seleccionado **Kernel** en el menú y luego **Restart**. Habiendo reiniciado el kernel, **corra todas las celdas**, esto lo puede hacer seleccionando **Cell** en el menú y luego **Run All**.

Por último, recuerde rellenar cualquier lugar donde diga `ESCRIBA SU CÓDIGO AQUÍ` o "ESCRIBA SU RESPUESTA AQUI", así como escribir su nombre y carné a continuación:

In [ ]:
NOMBRE = ""
CARNE = ""

---

## Preparación para este laboratorio
    
El presente laboratorio tiene como objetivos de aprendizaje:
    
1. Repasar los procedimientos básicos de estadística descriptiva
2. Reforzar los conceptos del teorema del límite central, intervalos de confianza y sus componentes
3. Aprender como calcular intervalos de confianza para la media manualmente y mediante la función `t.test()`

En este laboratorio utilizaremos el conjunto de datos de `aves`, junto con los paquetes `ggplot2`, `readr` y `dplyr`. Ya todos estos paquetes están descargados y disponibles en el entorno virtual, al igual que el conjunto de datos, por lo que no es necesario que descarguen o consigan ningún paquete o conjunto de datos adicional, todos ya están disponibles.

---

## Preparación del conjunto de datos

En este laboratorio, vamos a utilizar el conjunto de datos `aves`, el cual lo utilizamos anteriormente en el Laboratorio 1. Recordemos que este conjunto de datos contiene observaciones de dos especies distintas de aves *Chlorospingus pileatus* y *Catharis gracilirostris*. Carguemos este conjunto de datos a R mediante el comando `read_csv()`:

In [ ]:
library(readr)

aves <- read_csv("data/aves.csv")

Aunque ya este conjunto de datos lo hemos trabajado, veamos lo que contiene:

In [ ]:
summary(aves)

Recordemos que las variables `Localidad` y `Especie` son categóricas, por lo que ocupamos cambiarlas a factor en R. Además, vamos a cambiar el nombre de algunas variables que contienen un espacio, para poder trabajar con ellas más fácilmente. Procedamos a realizar esto:

In [ ]:
library(dplyr)
aves <- aves %>% mutate(Especie = factor(Especie),
                        Localidad = factor(Localidad)) %>% 
                 rename(Largo_pico = `Largo pico`, 
                        Alto_pico = `Alto pico`, 
                        Ancho_pico = `Ancho pico`)

Volvamos a hacer un `summary()` para verificar que todo funcionó de buena forma:

In [ ]:
summary(aves)

Parece que todo está bien! Notemos que en este conjunto de datos tenemos algunas variables que tienen valores faltantes, o NA's. Esto quiere decir que para algunas aves no se pudo obtener el valor de algunas variables. La decisión que necesitamos tomar ahora es que hacer con estos valores perdidos. En este laboratorio vamos a eliminarlos, para facilitar los análisis que haremos, pero esto no necesariamente va a ser el caso siempre. La eliminación de los NA's lo podemos hacer mediante el comando `na.omit()`, que elimina cualquier observación de un conjunto de datos que tenga un valor faltante. Procedamos a eliminar los valores faltantes y hacer un summary, de la siguiente forma:

In [ ]:
aves <- na.omit(aves)
summary(aves)

Como podemos observar, en este caso perdimos un total de 5 observaciones: 2 de *Catharus* y 3 de *Chlorospingus*. Aun así, también podemos notar que ya no hay NA's en el summary, lo que quiere decir que todos los datos con los que vamos a trabajar tienen observaciones para todas las variables. Por lo tanto, habiendo hecho los arreglos necesarios al conjunto de datos, podemos proceder a realizar los primeros estadísticos descriptivos, que haremos en la próxima sección.

---

## Estadísticos descriptivos

Como ya sabemos, el primer paso de cualquier análisis de datos va a ser un análisis descriptivo de los datos, mediante todo lo que hemos visto hasta el momento. En este caso, vamos a empezar por crear un histograma de cada una de las variables numéricas del conjunto de datos. El siguiente comando logra hacer todos los histogramas del conjunto de datos aves y, aunque es un poco más avanzado, se torna más fácil que crear cada histograma de forma individual:

In [ ]:
library(ggplot2)
library(tidyr)
aves %>% pivot_longer(c(-Localidad, -Especie), names_to = "Variable", values_to = "Valor") %>%
    ggplot(aes(x = Valor, fill = Variable)) +
    facet_wrap(~ Variable, scales = "free_x") +
    geom_histogram(bins = 8, colour = "black") +
    ylab("Frecuencia") +
    guides(fill = FALSE) +
    theme_bw()

Para analizar estos histogramas, recordemos que necesitamos analizar tres aspectos de estos: simetría, modalidad y valores extremos. Con respecto a la simetría, parece que todas las variables son relativamente simétricas, excepto por `Cola`, que parece tener una cola (valga la redundancia) larga hacia la derecha. Ahora, sobre la modalidad podemos ver que las variables de `Alto_pico` y `Tarso` tienen dos modas claramente definidas. Como hemos discutido en laboratorios anteriores, esto se puede deber a que hay un comportamiento distinto para estas variables dependiendo de la especie. Por último, la única distribución que tiene un posible valor extremos es la variable `Cola`, donde un ave tiene una cola de tamaño 0. 

En conclusión, si tuvieramos que hablar sobre variables que tienen una distribución similar a la normal, podríamos decir que `Largo_pico` y `Peso` son las variables que parecen tener una distribución normal, ya que las demás variables no son simétricas, o tienen dos modas claramente distintas, o tienen valores extremos. Aun así, para efectos del laboratorio, vamos a trabajar solo con la variable `Peso`.

Habiendo escogido la variable que queremos utilizar, procedamos al cálculo de los intervalos de confianza.

---

## Intervalo de confianza para la media

Recordemos que, hasta el momento, solamente hemos usado la **estadística descriptiva**, es decir, solamente hemos descrito como se comportan los datos de las muestras que hemos obtenido. Si queremos hacer generalizaciones, o inferencias, en la población con base en la muestra, podemos hacer uso de la **estadística inferencial**. Los intervalos de confianza son parte de la estadística inferencial ya que nos permiten generalizar el resultado puntual de un estimador en una muestra (como $\bar{x}$), para el parámetro desconocido de la población (como $\mu$).

De forma específica, el intervalo de confianza, es un rango de valores sobre el cual tenemos una cierta *confianza* de que contenga al parámetro. El cálculo del intervalo de confianza, en el caso específico de un promedio, es el siguiente:

$$
IC \text{ } 100(1-\alpha)\% = \bar{x} \pm t_{1-\alpha / 2, n - 1} \frac{s}{\sqrt{n}}
$$

donde $100(1 - \alpha)\%$ es el nivel de confianza, $\bar{x}$ es el promedio muestral, $s$ es la desviación estándar muestral, $n$ es el tamaño de muestra y $t_{1-\alpha / 2, n - 1}$ es el valor en una distribución t-Student con $n-1$ grados de libertad, que acumula $1 - \alpha/2$ de probabilidad hacia arriba. Este valor de la t-Student lo podemos visualizar de la siguiente forma, tomando como base la distribución normal:

<a title="M. W. Toews / CC BY (https://creativecommons.org/licenses/by/2.5)" href="https://commons.wikimedia.org/wiki/File:Standard_deviation_diagram.svg"><img width="256" alt="Standard deviation diagram" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Standard_deviation_diagram.svg/256px-Standard_deviation_diagram.svg.png"></a>

Este gráfico nos indica que la probabilidad de encontrar un valor que esté a una desviación estándar del promedio, ya sea hacia arriba o hacia abajo, es de un 68,2%. Más aún, la probabilidad de encontrar un valor que este a dos desviaciones estándar por encima o por debajo del promedio es de un 95,4%, mientras que la probabilidad de encontrar un valor a tres desviaciones estándar es de un 99,6%.

Como podemos ver de la fórmula, el único valor que depende del nivel de confianza ($\alpha$) es el valor de la t-Student. Esto se debe a que lo que estamos haciendo al aplicar la fórmula es lo mismo que discutimos con el gráfico anterior: estamos determinando la probabilidad de que el parámetro ($\mu$) se encuentre a una cierta cantidad ($t_{1-\alpha / 2, n - 1}$) de errores estándar ($s / \sqrt{n}$) del promedio estimado ($\bar{x}$). Por eso, cuando buscamos un intervalo de confianza del 95%, lo usual es multiplicar el error estándar por 2, dado que es el valor que acumula aproximadamente un 95% de probabilidad. Aun así, dado que tenemos R, no es necesario hacer estas aproximaciones y podemos calcular el valor directamente. Por último, recordemos que esta aproximación a una distribución t-Student/normal lo podemos hacer, independientemente de la distribución original de los datos, gracias al **Teorema del Límite Central**, por lo que solamente necesitamos un tamaño de muestra relativamente grande.

Habiendo repasado la fórmula y las nociones para el cálculo de los intervalos de confianza para la media, procedamos a calcular estos para las variables de interés.

---

### La variable `Peso`

Para el cálculo del intervalo de confianza, necesitamos las siguientes cuatro partes:

- $\bar{x}$, el promedio muestral
- $s$, la desviación estándar muestral
- $n$, el tamaño de la muestra
- $t_{1-\alpha / 2, n - 1}$, el valor de la t-Student

Obtengamos cada uno de estos valores en orden, empezando por el promedio:

In [ ]:
media.pe <- mean(aves$Peso) # Recordemos que ya quitamos todos los NA's
media.pe

Como podemos ver, esta es nuestra estimación del promedio muestral $\bar{x}$. Como sabemos, podemos interpretar este valor como que, en promedio, las aves de **nuestro conjunto de datos** pesan 20,17 g. Si queremos obtener un resultado para **la población de aves de ambas especies**, necesitamos calcular el intervalo de confianza. Por lo tanto, procedamos con el cálculo de la desviación estándar:

In [ ]:
sd.pe <- sd(aves$Peso)
sd.pe

En este caso, lo único que podemos decir es que la desviación estándar del peso de las aves de nuestra muestra es de 1.4 g. Al ser la desviación estándar, sus unidades de medición son iguales a las de la variable original, pero no podemos hacer más interpretaciones que eso. Ahora, si obtenemos el tamaño de muestra, podemos calcular el **error estándar**. El tamaño de muestras lo podemos obtener mediante la función `nrow()`, que cuenta la cantidad de filas de un conjunto de datos:

In [ ]:
n <- nrow(aves)
n

El resultado de $n$ es 92, es decir, tenemos un total de 92 aves en nuestro conjunto de datos. Este valor de $n$ va a ser igual para todos los intervalos de confianza para el promedio que hagamos con este conjunto de datos, dado que la cantidad de datos no cambiará de variable a variable si tenemos las mismas observaciones. Por lo tanto, lo único que nos falta es el valor de la t-Student, pero para poder hacer esto primero ocupamos hablar sobre la **confianza**. 

El concepto de **confianza** lo podemos entender como la probabilidad de que el intervalo que nosotros calculamos contenga al parámetro que queremos estimar. Por lo tanto, si tenemos una confianza de un 95%, estamos diciendo que la probabilidad de que el intervalo que calculamos contenga el parámetro deseado es de un 95%. Si recordamos, en la fórmula escribíamos la confianza como $100(1-\alpha)\%$, por lo que, para un 95% de confianza, podemos ver que el valor de $\alpha$ que ocupamos es el siguiente:

$$
100 (1 - \alpha) = 0.95 \Rightarrow 1 - \alpha = 0.95 \Rightarrow \alpha = 1 - 0.95 = 0.05
$$

Más adelante, veremos que este valor de $\alpha$ es conocido como la **significancia**, el cual usaremos para el tema de prueba de hipótesis. La siguiente es una lista de distintos niveles de confianza y el valor de $\alpha$ correspondiente:

| Confianza | $\alpha$ |
|:---------:|:--------:|
|    90%    |    0.1   |
|    95%    |   0.05   |
|    99%    |   0.01   |
|   99.9%   |   0.001  |

Por lo tanto, para poder obtener el valor de la t-Student, recordemos que la confianza es una probabilidad, por lo que tenemos que usar una función `q...()` para obtener el percentil correspondiente, en una t-Student, por lo que ocupamos usar la función `qt()`. Más aún, dado que tenemos $n-1$ grados de libertad, y queremos calcular la probabilidad acumulada hacia arriba, necesitamos agregar estos dos argumentos a la función, como se presenta a continuación:

In [ ]:
valor.t <- qt(1 - 0.05 / 2, n - 1)
valor.t

Note que este valor de t es muy cercano al valor de 2 del cuál habíamos hablado al principio de esta sección. Además, para un nivel de confianza fijo, podemos usar este valor de t para cualquier otro intervalo de confianza para una media con este conjunto de datos, dado que todos tienen el mismo tamaño de muestra y, por ende, grados de libertad.

Habiendo calculado este valor, podemos calcular el intervalo de confianza, juntanto todo lo que hemos obtenido hasta el momento:

In [ ]:
ic <- media.pe + c(-1, 1) * valor.t * sd.pe / sqrt(n)
ic

Lo más extraño del comando anterior es el uso de `c(-1, 1)`. Esto lo hacemos ya que R no tiene una función de sumar y restar al mismo tiempo, o "másmenos", por lo que necesitamos calcular dos números por separado, uno que sea el promedio *menos* el error estándar, y el promedio *más* el error estándar.

Al momento de interpretar este intervalo de confianza, que es tal vez lo más importante de este análisis, podemos decir que: **Con un 95% de confianza, el peso promedio poblacional de las aves de ambas especies está entre 19,88 y 20,46 g, aproximadamente**. Siempre es importante mencionar el nivel de confianza utilizado, que estamos haciendo conclusiones sobre la población y el parámetro, además de los límites propios del intervalo de confianza. Habiendo hecho esto, ya hicimos nuestra primer (pero no última) inferencia del curso!🎉

Todo este procedimiento del cálculo del intervalo de confianza de forma manual solamente lo vamos a hacer en este laboratorio. En la práctica esto lo podemos hacer, de forma automática, mediante el uso de la función `t.test()` la cual calcula, entre otras cosas, el intervalo de confianza para el promedio:

In [ ]:
t.test(aves$Peso)

Comparando el resultado del comando `t.test()` con el obtenido manualmente, podemos ver que la función nos devuelve más resultados que solamente el intervalo de confianza. Aun así, por ahora solo nos vamos a centrar en el intervalo de confianza, que se muestra en el resultado anterior como:

```
95 percent confidence interval:
 19.87815 20.45881
```

Si comparamos este resultado con el que obtuvimos manualmente, podemos ver que los dos resultados son básicamente idénticos, donde la única diferencia se debe a redondeos. Por lo tanto, de ahora en adelante, no es necesario calcular los intervalos de confianza para la media manualmente, sino que podemos utilizar la función `t.test()` e interpretar el resultado directamente de ahí.

Un comentario con respecto a la función `t.test()` es que, por default, hace los intervalos de confianza con un 95% de confianza y, para poder cambiar este nivel de confianza, lo podemos hacer mediante el argumento `conf.level`, que toma un número entre 0 y 1, correspondiente al nivel de confianza. Por lo tanto, para obtener un intervalo de confianza con un 98% de confianza, necesitamos agregar el argumento `conf.level = 0.98` a la función `t.test()`.

---

### Ejercicio (6 pts.)

Para la variable `Juanete`:

1. Calcule el intervalo de confianza para el promedio con un 80, 90, 95 y 99% de confianza e interprételos (puede usar la función `t.test()`). 
2. ¿Qué pasa con los intervalos de confianza conforme aumenta la confianza: se vuelven más estrechos o más anchos? 
3. ¿Parece haber una diferencia importante entre los límites del intervalo de confianza cuando se utiliza un 80% o un 99% de confianza? 

In [ ]:
# ESCRIBA SU CÓDIGO AQUÍ
testthat::fail() # No respuesta - elimine esta línea si escribió una respuesta

ESCRIBA SU RESPUESTA AQUI

---

## <span style="color: blue">Entrega del laboratorio</span> <a class="tocSkip"> 

Para hacer entrega de este laboratorio, debe descargarlo presionando en **File**, luego **Download as** y, por último, **Notebook (.ipynb)**, en el menú superior, debajo del logo de Jupyter. También puede descargarlo directamente desde el botón de **Download** en la barra con íconos para acceso rápido.

Luego de haber localizado el archivo descargado, debe cambiar el nombre del archivo descargado por su número de carné, de forma que, si su carné es **B12345**, su archivo debería llamarse `B12345.ipynb`. Una vez le haya cambiado el nombre al archivo, lo puede subir a Mediación Virtual, en el apartado correspondiente a este laboratorio.

<a href="https://creativecommons.org/licenses/by-sa/4.0/">
    <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-sa.png" width="88" height="31" />
</a>